#Imports e dados

In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import MinMaxScaler
from scipy import stats as st
from scipy.stats import f, f_oneway

In [4]:
df_atend = pd.read_csv('dados_atendimentos.csv')
df_renda = pd.read_csv('dados_renda_municipios_2010.csv')

In [5]:
df_atend.head(5)

,idEntrada,state,usuarioOnline,statusSaida,paginaSaida,avaliacao,dataHoraEntrada,dataHoraSaida,dataHoraErro,tempoAcesso,codigo,erro
0,1,CE,1,80%,1,positiva,18/05/2021 17:57,18/05/2021 18:00,18/05/2021 17:59,173,550,"""NO MOMENTO ESTE SERIVCO ESTA INDISPONIVEL. PO..."
1,2,PE,0,40%,3,negativa,18/05/2021 17:57,18/05/2021 18:01,18/05/2021 18:00,212,550,"""ERRO NA VALIDACAO DE DADOS COMPLEMENTARES"""
2,3,CE,0,60%,2,negativa,18/05/2021 17:57,18/05/2021 18:04,18/05/2021 18:03,427,500,"""INTERNAL SERVER ERROR"""
3,4,PR,0,20%,4,negativa,18/05/2021 17:57,18/05/2021 18:01,18/05/2021 18:00,248,550,"""CPF -CNPJ CLIENTE NAO CADASTRADO"""
4,5,TO,0,20%,4,positiva,18/05/2021 17:57,18/05/2021 18:01,18/05/2021 18:02,256,500,"""OPS! NÃO FOI POSSIVEL CONSULTAR O SALDO. TENT..."


In [6]:
df_renda.head(5)

,state,city,RDPC
0,GO,Abadia de Goiás/GO,574.96
1,MG,Abadia dos Dourados/MG,596.18
2,GO,Abadiânia/GO,519.87
3,PA,Abaetetuba/PA,293.01
4,MG,Abaeté/MG,707.24


# Atividade 3 – Projeto relacionamento com clientes

Você é um cientista de dados de um Banco no Brasil. Na etapa de atendimento, existe um espaço para os clientes realizarem uma avaliação do processo. No banco de dados “avaliações_clientes.csv” temos algumas informações sobre acessos, resultados e avaliações.
Utilizar o arquivo checkpoint3_sem2.ipynb para obter os dados do Banco e carregar os dados do censo de 2010. Faça e responda as seguintes perguntas:


1 – Na tabela do censo de 2010, obter o índice de Gini por UF (state) (criar um dataframe como
df_gini_estado) e realizar o join das tabelas de atendimento e df_gini_estado.

Função de Gini

In [7]:
def def_gini(array):
    """Calculate the Gini coefficient of a numpy array."""
    array = np.array(array)
    array = array.flatten()
    if np.amin(array) < 0:
        array -= np.amin(array)
    array += 0.0000001  # Evite dividir por 0
    array = np.sort(array)
    index = np.arange(1, array.shape[0] + 1)
    n = array.shape[0]
    return ((np.sum((2 * index - n - 1) * array)) / (n * np.sum(array)))


df_gini_estado = df_renda.groupby('state')['RDPC'].apply(def_gini).reset_index()
df_gini_estado.columns = ['state', 'gini_coe']
df_gini_estado.head(5)

,state,gini_coe
0,AC,0.195495
1,AL,0.140823
2,AM,0.180262
3,AP,0.177035
4,BA,0.160275


Join de tabelas

In [8]:
atend_gini = pd.merge(df_atend, df_gini_estado, on='state', how='inner')
atend_gini.head(5)

,idEntrada,state,usuarioOnline,statusSaida,paginaSaida,avaliacao,dataHoraEntrada,dataHoraSaida,dataHoraErro,tempoAcesso,codigo,erro,gini_coe
0,1,CE,1,80%,1,positiva,18/05/2021 17:57,18/05/2021 18:00,18/05/2021 17:59,173,550,"""NO MOMENTO ESTE SERIVCO ESTA INDISPONIVEL. PO...",0.132551
1,3,CE,0,60%,2,negativa,18/05/2021 17:57,18/05/2021 18:04,18/05/2021 18:03,427,500,"""INTERNAL SERVER ERROR""",0.132551
2,25,CE,0,40%,3,negativa,18/05/2021 17:57,18/05/2021 18:00,18/05/2021 18:00,165,550,"""NO MOMENTO ESTE SERIVCO ESTA INDISPONIVEL. PO...",0.132551
3,74,CE,0,100%,0,n/ avaliado,18/05/2021 17:57,18/05/2021 18:00,18/05/2021 17:59,155,500,"""ERRO AO EXECUTAR SERVICO: REQUESTCREDITSERVICE""",0.132551
4,99,CE,0,60%,2,n/ avaliado,18/05/2021 17:57,18/05/2021 18:01,18/05/2021 17:59,212,550,"""NAO ENCONTROU DADOS PARA A SELECAO""",0.132551


In [9]:
#verificando valores nulos
missing_data = atend_gini.isna().sum()
print("Valores ausentes por coluna:\n", missing_data)


Valores ausentes por coluna:
 idEntrada          0
state              0
usuarioOnline      0
statusSaida        0
paginaSaida        0
avaliacao          0
dataHoraEntrada    0
dataHoraSaida      0
dataHoraErro       0
tempoAcesso        0
codigo             0
erro               0
gini_coe           0
dtype: int64


In [10]:
# Dropando "não avaliado" das avaliações
atend_gini_aval2 = atend_gini[atend_gini['avaliacao'] != 'n/ avaliado']
tipos_avaliacao = atend_gini_aval2['avaliacao'].unique()
print("Tipos de registros na coluna avaliacao:", tipos_avaliacao)

Tipos de registros na coluna avaliacao: ['positiva' 'negativa']


In [11]:
#criando outros data frames:
df_aval_tmpacess = atend_gini_aval2[['avaliacao', 'tempoAcesso']].reset_index(drop=True)
df_aval_gini = atend_gini_aval2[['avaliacao', 'gini_coe']].reset_index(drop=True)
df_aval_tmpacess3 = atend_gini[['avaliacao', 'tempoAcesso']].reset_index(drop=True)
df_aval_gini3 = atend_gini[['avaliacao', 'gini_coe']].reset_index(drop=True)
#df_aval_tmpacess.head(5)
#df_aval_gini.head(5)
#df_aval_tmpacess3.head(5)
#df_aval_gini3.head(5)

2 – Realizar o teste t para os itens abaixo:

a) Avaliação (Positiva e Negativa) e Tempo de Acesso.

In [12]:
#group by
df_aval_tmpacess.groupby('avaliacao') \
                     .agg(media_tempo = pd.NamedAgg('tempoAcesso', 'mean'),
                          dp_tempo = pd.NamedAgg('tempoAcesso', 'std'),
                          n = pd.NamedAgg('tempoAcesso', 'count')) \
                     .reset_index()

,avaliacao,media_tempo,dp_tempo,n
0,negativa,246.870588,113.674072,510
1,positiva,252.876494,107.700718,251


In [13]:
gr1a = df_aval_tmpacess[df_aval_tmpacess['avaliacao'] == 'negativa']['tempoAcesso']
gr2a = df_aval_tmpacess[df_aval_tmpacess['avaliacao'] == 'positiva']['tempoAcesso']

In [14]:
st.ttest_ind(a=gr1a, b=gr2a, alternative='two-sided')

TtestResult(statistic=-0.6970955841932512, pvalue=0.48595643103026165, df=759.0)

- Hipótese nula: Se a avaliação não é afetada pelo tempo de acesso
- Hipótese alternativa: Se a avaliação é afetada pelo tempo de acesso

- Estimador: média

- Nível de confiança: 95%

- Valor estatistica teste t: -0.6970955841932512

- Rejeitar ou não rejeitar: Há evidências que a hipótese Nula NÃO deve ser rejeitada.

b) Avaliação (Positiva e Negativa) e GINI.

In [15]:
#group by
df_aval_gini.groupby('avaliacao') \
                     .agg(media_gini = pd.NamedAgg('gini_coe', 'mean'),
                          dp_gini = pd.NamedAgg('gini_coe', 'std'),
                          n = pd.NamedAgg('gini_coe', 'count')) \
                     .reset_index()

,avaliacao,media_gini,dp_gini,n
0,negativa,0.158849,0.027977,510
1,positiva,0.158473,0.027496,251


In [16]:
gr1g = df_aval_gini[df_aval_gini['avaliacao'] == 'negativa']['gini_coe']
gr2g = df_aval_gini[df_aval_gini['avaliacao'] == 'positiva']['gini_coe']

In [17]:
st.ttest_ind(a=gr1g, b=gr2g, alternative='two-sided')

TtestResult(statistic=0.17533257796804042, pvalue=0.8608650776872668, df=759.0)

Hipótese nula: Se a avaliação não é afetada pelo Índice de Gini.

Hipótese alternativa: Se a avaliação é afetada pelo Índice de Gini.

Estimador: média

Nível de confiança: 95%

Valor estatistica teste t: 0.17533257796804042

Rejeitar ou não rejeitar: Há evidências que a hipótese Nula NÃO deve ser rejeitada.

3 – Realizar o teste F para os itens abaixo:

a) Avaliação e Tempo de Acesso.

In [18]:
#group by
df_aval_tmpacess.groupby('avaliacao') \
                     .agg(media_tempo = pd.NamedAgg('tempoAcesso', 'mean'),
                          dp_tempo = pd.NamedAgg('tempoAcesso', 'std'),
                          n = pd.NamedAgg('tempoAcesso', 'count')) \
                     .reset_index()

,avaliacao,media_tempo,dp_tempo,n
0,negativa,246.870588,113.674072,510
1,positiva,252.876494,107.700718,251


In [19]:
gr1aF = df_aval_tmpacess[df_aval_tmpacess['avaliacao'] == 'negativa']['tempoAcesso']
gr2aF = df_aval_tmpacess[df_aval_tmpacess['avaliacao'] == 'positiva']['tempoAcesso']

In [20]:
f_valor = np.var(gr1aF, ddof=1) / np.var(gr2aF, ddof=1)
f_valor

1.1140011410206616

In [21]:
gl_a = len(gr1aF) - 1
gl_b = len(gr2aF) - 1
gl_a, gl_b

(509, 250)

In [22]:
def f_p_value(f_statistic, df_n, df_d, test_type):

  '''test_type: greater, less, two.sided'''

  if test_type == 'greater':
    return 1 - f.cdf(f_valor, df_n, df_d)
  elif test_type == 'less':
    return f.cdf(f_valor, df_n, df_d)
  elif test_type == 'two.sided':
    p1 = f.cdf(f_valor, df_n, df_d)
    p2 = 1 - f.cdf(f_valor, df_n, df_d)
    return np.min([p1, p2])*2
  else:
    raise TypeError("'test_type' only accept options: 'greater', 'less' or 'two.sided'")

In [23]:
p_value = f_p_value(f_valor, gl_a, gl_b, 'two.sided')

In [24]:
f_valor, p_value

(1.1140011410206616, 0.33250259900542023)

Hipótese nula: Se a avaliação não é afetada pelo tempo de acesso.

Hipótese alternativa: Se a avaliação é afetada pelo tempo de acesso.

Estimador: média

Nível de confiança: 95%

Valor estatistica teste f: 1.1140011410206616

Rejeitar ou não rejeitar: Há evidências que a hipótese Nula NÃO deve ser rejeitada.

b) Avaliação e GINI.

In [25]:
gr1gF = df_aval_gini[df_aval_gini['avaliacao'] == 'negativa']['gini_coe']
gr2gF = df_aval_gini[df_aval_gini['avaliacao'] == 'positiva']['gini_coe']

In [26]:
f_valorG = np.var(gr1gF, ddof=1) / np.var(gr2gF, ddof=1)
f_valorG

1.0352965767700752

In [27]:
gl_ag = len(gr1aF) - 1
gl_bg = len(gr2aF) - 1
gl_ag, gl_bg

(509, 250)

In [28]:
def f_p_value(f_statistic, df_n, df_d, test_type):

  '''test_type: greater, less, two.sided'''

  if test_type == 'greater':
    return 1 - f.cdf(f_valorG, df_n, df_d)
  elif test_type == 'less':
    return f.cdf(f_valorG, df_n, df_d)
  elif test_type == 'two.sided':
    p1 = f.cdf(f_valorG, df_n, df_d)
    p2 = 1 - f.cdf(f_valorG, df_n, df_d)
    return np.min([p1, p2])*2
  else:
    raise TypeError("'test_type' only accept options: 'greater', 'less' or 'two.sided'")

In [29]:
p_valueG = f_p_value(f_valorG, gl_ag, gl_bg, 'two.sided')

In [30]:
f_valorG, p_valueG

(1.0352965767700752, 0.7608905704855671)

Hipótese nula: Se a avaliação não é afetada pelo índice de Gini

Hipótese alternativa: Se a avaliação é afetada pelo índice de Gini

Estimador: média

Nível de confiança: 95%

Valor estatistica teste f: 1.0352965767700752

Rejeitar ou não rejeitar: Há evidências que a hipótese Nula NÃO deve ser rejeitada.

4 – Realizar o teste de Qui-Quadrado para:

a) Avaliação e código de erro (utilizando somente positivos e negativos)


In [34]:
atend_gini_aval2

,idEntrada,state,usuarioOnline,statusSaida,paginaSaida,avaliacao,dataHoraEntrada,dataHoraSaida,dataHoraErro,tempoAcesso,codigo,erro,gini_coe
0,1,CE,1,80%,1,positiva,18/05/2021 17:57,18/05/2021 18:00,18/05/2021 17:59,173,550,"""NO MOMENTO ESTE SERIVCO ESTA INDISPONIVEL. PO...",0.132551
1,3,CE,0,60%,2,negativa,18/05/2021 17:57,18/05/2021 18:04,18/05/2021 18:03,427,500,"""INTERNAL SERVER ERROR""",0.132551
2,25,CE,0,40%,3,negativa,18/05/2021 17:57,18/05/2021 18:00,18/05/2021 18:00,165,550,"""NO MOMENTO ESTE SERIVCO ESTA INDISPONIVEL. PO...",0.132551
5,141,CE,1,80%,1,negativa,18/05/2021 17:57,18/05/2021 18:02,18/05/2021 18:02,324,550,"""NO MOMENTO ESTE SERIVCO ESTA INDISPONIVEL. PO...",0.132551
6,144,CE,1,80%,1,negativa,18/05/2021 17:57,18/05/2021 17:58,18/05/2021 17:57,58,550,"""NAO ENCONTROU DADOS PARA A SELECAO""",0.132551
...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,817,PA,1,80%,1,positiva,18/05/2021 17:57,18/05/2021 18:01,18/05/2021 18:00,217,500,"""ERRO AO EXECUTAR SERVICO: REQUESTCREDITSERVICE""",0.208619
995,839,PA,1,80%,1,negativa,18/05/2021 17:57,18/05/2021 18:01,18/05/2021 17:59,247,500,"""INTERNAL SERVER ERROR""",0.208619
997,876,PA,0,40%,3,negativa,18/05/2021 17:57,18/05/2021 18:05,18/05/2021 18:03,474,500,"""OPS! NÃO FOI POSSIVEL CONSULTAR O SALDO. TENT...",0.208619
998,936,PA,1,80%,1,negativa,18/05/2021 17:57,18/05/2021 18:00,18/05/2021 18:00,171,500,"""DESCULPE TIVEMOS UM PROBLEMA E NAO CONSEGUIMO...",0.208619


In [36]:
atend_gini_aval2['codigo'].unique()

array([550, 500])

In [53]:
crosstab = pd.crosstab(atend_gini_aval2['avaliacao'], atend_gini_aval2['codigo'])
crosstab

codigo,500,550
avaliacao,,
negativa,238,272
positiva,130,121


## Teste de Aderência


In [54]:
frequencia_observada = np.array([238, 130, 272, 121 ])

In [55]:
p_esperada = np.array([0.40, 0.10, 0.40, 0.10])

In [56]:
frequencia_esperada = np.sum(frequencia_observada) * p_esperada
frequencia_esperada

array([304.4,  76.1, 304.4,  76.1])

In [57]:
st.chisquare(f_obs=frequencia_observada, f_exp=frequencia_esperada)

Power_divergenceResult(statistic=82.6005256241787, pvalue=8.494164829847333e-18)

## Graus de liberdade

In [58]:
crosstab = pd.crosstab(atend_gini_aval2['avaliacao'], atend_gini_aval2['codigo'])

In [59]:
linhas, colunas = crosstab.shape
print(f"Número de linhas (r): {linhas}")
print(f"Número de colunas (c): {colunas}")

Número de linhas (r): 2
Número de colunas (c): 2


In [60]:
graus_liberdade = (linhas - 1) * (colunas - 1)
print(f"Graus de liberdade: {graus_liberdade}")

Graus de liberdade: 1


## Teste de homogeneidade

In [61]:
crosstab = pd.crosstab(atend_gini_aval2['avaliacao'], atend_gini_aval2['codigo'])
crosstab

codigo,500,550
avaliacao,,
negativa,238,272
positiva,130,121


In [66]:
chi, p_value, gl, t_esp = st.chi2_contingency(crosstab, correction=False)

In [67]:
chi, p_value

(1.7699982714714046, 0.18338229295716996)

In [69]:
gl, t_esp

(1,
 array([[246.62286465, 263.37713535],
        [121.37713535, 129.62286465]]))

## Teste de Independência

In [70]:
tabela = pd.crosstab(atend_gini_aval2['avaliacao'], atend_gini_aval2['codigo'])
tabela

codigo,500,550
avaliacao,,
negativa,238,272
positiva,130,121


In [71]:
chi, p_value, gl, t_esp = st.chi2_contingency(tabela, correction=False)

In [72]:
chi, p_value

(1.7699982714714046, 0.18338229295716996)

Hipótese nula: Se a avaliação não é afetada pelo codigo de erro

Hipótese alternativa: Se a avaliação é afetada pelo codigo de erro

Estimador: qui-quadrado

Nível de confiança: 95%

Graus de liberdade: 1

Rejeitar ou não rejeitar: Há evidências que a hipótese Nula NÃO deve ser rejeitada.

b) Avaliação e código de erro (utilizando positivos, negativos e não avaliado)


In [74]:
atend_gini['codigo'].unique()

array([550, 500])

In [75]:
crosstab = pd.crosstab(atend_gini['avaliacao'], atend_gini['codigo'])
crosstab

codigo,500,550
avaliacao,,
n/ avaliado,99,140
negativa,238,272
positiva,130,121


## Teste de Aderência

In [76]:
frequencia_observada = np.array([99, 238, 130, 140, 272, 121 ])

In [77]:
p_esperada = np.array([0.10, 0.30, 0.10, 0.10, 0.30, 0.10])

In [78]:
frequencia_esperada = np.sum(frequencia_observada) * p_esperada
frequencia_esperada

array([100., 300., 100., 100., 300., 100.])

In [79]:
st.chisquare(f_obs=frequencia_observada, f_exp=frequencia_esperada)

Power_divergenceResult(statistic=44.846666666666664, pvalue=1.5588334148673004e-08)

## Graus de liberdade

In [80]:
linhas, colunas = crosstab.shape
print(f"Número de linhas (r): {linhas}")
print(f"Número de colunas (c): {colunas}")

Número de linhas (r): 3
Número de colunas (c): 2


In [81]:
graus_liberdade = (linhas - 1) * (colunas - 1)
print(f"Graus de liberdade: {graus_liberdade}")

Graus de liberdade: 2


## Teste de homogeneidade

In [83]:
crosstab = pd.crosstab(atend_gini['avaliacao'], atend_gini['codigo'])
crosstab

codigo,500,550
avaliacao,,
n/ avaliado,99,140
negativa,238,272
positiva,130,121


In [87]:
chi, p_value, gl, t_esp = st.chi2_contingency(crosstab, correction=False)

In [88]:
chi, p_value

(5.289891400300267, 0.07100920942436537)

In [89]:
gl, t_esp

(2,
 array([[111.613, 127.387],
        [238.17 , 271.83 ],
        [117.217, 133.783]]))

## Teste de Independência

In [91]:
tabela = pd.crosstab(atend_gini['avaliacao'], atend_gini['codigo'])
tabela

codigo,500,550
avaliacao,,
n/ avaliado,99,140
negativa,238,272
positiva,130,121


In [92]:
chi, p_value, gl, t_esp = st.chi2_contingency(tabela, correction=False)

In [93]:
chi, p_value

(5.289891400300267, 0.07100920942436537)

Hipótese nula: Se a avaliação não é afetada pelo codigo de erro

Hipótese alternativa: Se a avaliação é afetada pelo codigo de erro

Estimador: qui-quadrado

Nível de confiança: 95%

Graus de liberdade: 2

Rejeitar ou não rejeitar: Há evidências que a hipótese Nula NÃO deve ser rejeitada.

5 – Obter a correlação entre Gini e Tempo de acesso. A correlação é fraca, moderada ou forte?

In [32]:
correlacao = atend_gini['tempoAcesso'].corr(atend_gini['gini_coe'])
print(f"Correlação entre tempoAcesso e gini_coe: {correlacao}")

Correlação entre tempoAcesso e gini_coe: 0.03691629889076111


A correlaçãop entre Gini e tempo de acesso (0,036) é considerada fraca, apesar de positiva, indicando que as váriáveis são provavelemnte independentes. A correlação é muito próxia de zero.

In [33]:
spearman_corr = atend_gini['tempoAcesso'].corr(atend_gini['gini_coe'], method='spearman')
print(f"Correlação de Spearman entre tempoAcesso e gini_coe: {spearman_corr}")

Correlação de Spearman entre tempoAcesso e gini_coe: 0.06609216869253286


Testamos também a correlação de Spearman que é usada pra dados não lineares e ainda assim, continua um correlação positiva muito fraca (0,066).